# LLM ChatBot - Complete Workflow

This notebook demonstrates the complete pipeline for building a ChatGPT-like LLM system on Paperspace.

## Steps:
1. Setup and verification
2. Dataset preparation
3. Model fine-tuning with LoRA
4. Inference and testing
5. Deployment (Gradio & API)

**Estimated Time**: 2-4 hours (depending on dataset size)

## 1. Setup and Environment Verification

In [1]:
# Check GPU availability
import torch
import sys

print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU count: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        gpu_memory = torch.cuda.get_device_properties(i).total_memory / 1024**3
        print(f"  Memory: {gpu_memory:.2f} GB")
else:
    print("WARNING: No GPU detected! Training will be very slow.")

Python version: 3.11.7 (main, Dec  8 2023, 18:56:58) [GCC 11.4.0]
PyTorch version: 2.1.1+cu121
CUDA available: True
CUDA version: 12.1
GPU count: 1
GPU 0: NVIDIA RTX A4000
  Memory: 15.72 GB


In [3]:
!ls notebooks/

In [4]:
# Set up environment variables
import os

os.environ["HF_HOME"] = "./cache"
os.environ["TRANSFORMERS_CACHE"] = "./cache"

# Optional: Set your HuggingFace token if using gated models (e.g., Llama 2)
# os.environ["HF_TOKEN"] = "your_token_here"

print("✅ Environment configured")

✅ Environment configured


## 2. Dataset Preparation

We'll use the Alpaca dataset for instruction fine-tuning. Start with a small subset for testing.

In [5]:
from prepare_dataset_notebook import prepare_dataset

# Prepare dataset (start with 1000 samples for testing)
dataset = prepare_dataset(
    dataset_name="tatsu-lab/alpaca",
    template_name="alpaca",
    max_samples=1000,  # Use 1000 for quick test, None for full dataset
    output_dir="./data/processed",
    preview=True
)

print(f"\n✅ Dataset prepared!")
print(f"Train samples: {len(dataset['train'])}")
print(f"Validation samples: {len(dataset['validation'])}")

ModuleNotFoundError: No module named 'prepare_dataset_notebook'

In [ ]:
# Inspect a sample
print("Sample training example:")
print("=" * 80)
print(dataset['train'][0]['text'][:500])
print("...")

## 3. Model Fine-Tuning with LoRA

Train the model using QLoRA for memory efficiency. This will take the most time (~15 mins for 1000 samples).

In [ ]:
from train_lora import LLMTrainer

# Initialize trainer
trainer = LLMTrainer(
    model_name="mistralai/Mistral-7B-v0.1",  # Or "TinyLlama/TinyLlama-1.1B-Chat-v1.0" for faster testing
    dataset_path="./data/processed",
    output_dir="./models/checkpoints",
    final_model_dir="./models/final",
    load_in_4bit=True,  # Use 4-bit quantization
    lora_r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    learning_rate=2e-4,
    num_epochs=3,  # Reduce to 1 for quick test
    batch_size=4,
    gradient_accumulation_steps=4,
    max_length=2048,
    use_wandb=False,  # Set to True if you have W&B API key
)

print("Trainer initialized!")

In [ ]:
# Load model and tokenizer
trainer.load_model_and_tokenizer()
print("\n✅ Model and tokenizer loaded!")

In [ ]:
# Configure LoRA
trainer.configure_lora()
print("\n✅ LoRA configured!")

In [ ]:
# Load dataset
trainer.load_dataset()
print("\n✅ Dataset loaded!")

In [ ]:
# Start training (this will take time!)
print("Starting training...")
print("This may take 15-30 minutes for 1000 samples, 4-8 hours for full dataset.")
print("You can monitor progress in the output below.\n")

result = trainer.train()

print("\n" + "=" * 80)
print("🎉 Training complete!")
print("=" * 80)

## 4. Inference and Testing

Now let's test the fine-tuned model!

In [ ]:
from inference import create_chatbot

# Create chatbot with fine-tuned model
bot = create_chatbot(
    model_name="mistralai/Mistral-7B-v0.1",
    adapter_path="./models/final",  # LoRA adapter
    load_in_4bit=True
)

print("✅ ChatBot loaded and ready!")

In [ ]:
# Test single-turn generation
response = bot.chat(
    "What is machine learning?",
    max_new_tokens=256,
    temperature=0.7
)

print("User: What is machine learning?")
print(f"\nAssistant: {response}")

In [ ]:
# Test multi-turn conversation
bot.reset_conversation()

questions = [
    "Hello! Can you help me understand neural networks?",
    "What are the main components?",
    "Can you give a simple example?",
]

for question in questions:
    response = bot.chat(question, max_new_tokens=200)
    print(f"\nUser: {question}")
    print(f"Assistant: {response}")
    print("-" * 80)

In [ ]:
# View conversation history
history = bot.get_conversation_history()
print(f"\nConversation length: {len(history)} messages")

for i, msg in enumerate(history):
    print(f"{i+1}. {msg['role']}: {msg['content'][:100]}...")

## 5. Interactive Testing

Try different generation parameters:

In [ ]:
# Test with different temperatures
prompt = "Write a creative story about AI"

print("Low temperature (more focused):")
print("=" * 80)
response_low = bot.generate(prompt, temperature=0.3, max_new_tokens=150)
print(response_low)

print("\n\nHigh temperature (more creative):")
print("=" * 80)
response_high = bot.generate(prompt, temperature=1.2, max_new_tokens=150)
print(response_high)

## 6. Safety and Content Filtering

Add safety features to your chatbot:

In [ ]:
from safety_utils import ContentFilter, UsageLogger

# Initialize safety features
content_filter = ContentFilter(
    max_input_length=2048,
    blocked_words=[],  # Add words to block
)

usage_logger = UsageLogger()

print("✅ Safety features initialized")

In [ ]:
# Safe chat function
def safe_chat(user_input, bot, filter, logger):
    """Chat with safety checks"""
    import time
    
    # Validate input
    is_valid, error = filter.validate_input(user_input)
    if not is_valid:
        print(f"❌ Error: {error}")
        return None
    
    # Generate response
    start_time = time.time()
    response = bot.chat(user_input)
    duration = (time.time() - start_time) * 1000
    
    # Log interaction
    filter.log_interaction(user_input, response)
    logger.log_request(
        endpoint="chat",
        tokens_used=len(response.split()),
        duration_ms=duration
    )
    
    return response

# Test safe chat
response = safe_chat(
    "Tell me about quantum computing",
    bot,
    content_filter,
    usage_logger
)

print(f"Response: {response}")

## 7. Launch Web Interface

Deploy your chatbot with Gradio:

In [ ]:
from app_gradio import launch_gradio

# Launch Gradio interface
# This will create a public link if share=True
launch_gradio(
    model_name="mistralai/Mistral-7B-v0.1",
    adapter_path="./models/final",
    load_in_4bit=True,
    share=True,  # Creates public link (optional)
    port=7860
)

# Note: This will block the notebook. Stop the cell to continue.

## 8. Launch API Server (Alternative)

Run FastAPI server for REST API access:

In [ ]:
# Don't run this in the same notebook as Gradio
# Use this in a separate notebook or terminal

# from app_api import launch_api
# 
# launch_api(
#     host="0.0.0.0",
#     port=8000
# )

## 9. Test API (if running)

Test the API endpoints:

In [ ]:
import requests
import os

# Set your API key
API_KEY = os.getenv("API_SECRET_KEY", "your-secret-key-change-this")
API_URL = "http://localhost:8000"

# Test health endpoint
response = requests.get(f"{API_URL}/health")
print("Health check:")
print(response.json())

# Test chat endpoint
headers = {"Authorization": f"Bearer {API_KEY}"}
data = {
    "message": "What is deep learning?",
    "max_tokens": 256,
    "temperature": 0.7
}

response = requests.post(
    f"{API_URL}/chat",
    headers=headers,
    json=data
)

print("\nChat response:")
print(response.json())

## 10. Summary and Next Steps

✅ You've completed:
1. Environment setup and verification
2. Dataset preparation
3. Model fine-tuning with LoRA/QLoRA
4. Inference and testing
5. Safety features implementation
6. Web interface deployment

### Next Steps:

1. **Train on more data**: Increase `max_samples` to use full dataset
2. **Experiment with models**: Try different base models (Llama 2, Phi-2)
3. **Tune hyperparameters**: Adjust LoRA rank, learning rate, epochs
4. **Add custom dataset**: Prepare your own instruction dataset
5. **Deploy to production**: Use Docker for deployment
6. **Monitor performance**: Set up W&B or TensorBoard
7. **Export models**: Convert to GGUF for llama.cpp

### Resources:
- README: `README_LLM_CHATBOT.md`
- Configuration: `config_training.yaml`, `config_inference.yaml`
- Logs: `./logs/training/`, `./logs/inference/`
- Models: `./models/final/`